In [4]:
import boto3
import time

client = boto3.client('athena',region_name='us-east-1')
s3_input = 's3://mist-secor-production/cv-sle-multipartition/'

# bucket = 'mist-{ENV}-athena'.format(ENV=env)
# s3_ouput = 's3://%s/results/' % bucket
# download_path = os.curdir + '/../downloads/'

s3_output = 's3://aws-athena-query-results-660610034966-us-east-1/test'

database = 'secorapp-production'
table = 'cv_sle_multipartition'
#Function for starting athena query
def run_query(query, database, s3_output):
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    print('Execution ID: ' + response['QueryExecutionId'])
    return response

In [5]:
def create_query(minute, days, siteid):
    query = """
    SELECT sum(duration) AS duration,
             site_id ,
             date_trunc('minute', date_add('minute', -(minute(time)%{minute}), time)) AS hour,
             count(distinct client_wcid) as counts,
             count(distinct CASE WHEN classifier != '' THEN client_wcid ELSE NULL END) as impacted_counts,
             SUM(CASE WHEN classifier != '' THEN duration ELSE 0 END) failure,
             SUM(CASE WHEN classifier = '' THEN duration ELSE 0 END) success,
             SUM(CASE WHEN classifier = 'association' THEN duration ELSE 0 END) association,
             SUM(CASE WHEN classifier = 'authentication' THEN duration ELSE 0 END) authentication,
             SUM(CASE WHEN classifier = 'dhcp' THEN duration ELSE 0 END) dhcp,
             SUM(CASE WHEN classifier = 'unclear' THEN duration ELSE 0 END) unclear
    FROM "secorapp-production"."cv_sle_multipartition"
    WHERE time > date_add('day',-{days},now())
            AND site_id='{siteid}'
            AND metric='connect-failures'
    GROUP BY  2, 3
    ORDER BY  3 asc
    """
    query = query.format(minute=minute, days=days, siteid=siteid)
    return query

# Wait for the file to be ready
def fetch_query_result(local_file_name, res):
    _N_iteration = 20
    _waiting_time = 5 #in Second
    s3 = boto3.resource('s3')
    Bucket = 'aws-athena-query-results-660610034966-us-east-1'
    my_bucket = s3.Bucket(Bucket)
    key = "test/{EXEC_ID}.csv".format(EXEC_ID=res['QueryExecutionId'])
    for _ in range(_N_iteration):
        objs = list(my_bucket.objects.filter(Prefix=key))
        if len(objs) > 0 and objs[0].key == key:
            my_bucket.download_file(key,Filename=local_file_name)
            return
        else:
            time.sleep(_waiting_time)
    my_bucket.download_file(key,Filename=local_file_name)

In [6]:
minute = 60
days = 1
siteid = '67970e46-4e12-11e6-9188-0242ac110007'

query = create_query(minute=minute, days=days, siteid=siteid)
#query = 'SELECT id, svis[0] FROM "secorapp-staging"."ap_stats_analytics" limit 10'
res = run_query(query, database, s3_output)
local_file_name = '/tmp/{siteid}_v2.06.csv'.format(siteid=siteid)
fetch_query_result(local_file_name, res)

Execution ID: 72d6ab2e-16ad-481a-b390-3b297d0b613a
